In [6]:
import re
import pandas as pd
import numpy as np
import pyvista as pv

In [7]:
# Import CSV (lidar_dist_meters, lidar_angle_deg, servo_angle_deg)

df = pd.read_csv('/Users/anxiankhoo/Documents/lidar/lidardatafile.csv')
df.columns = ["lidar_angle_deg", "lidar_dist_mm", "quality", "servo_angle_deg"]
df

,lidar_angle_deg,lidar_dist_mm,quality,servo_angle_deg
0,0.53,0.0,0,0.0
1,0.88,0.0,0,0.0
2,1.24,0.0,0,0.0
3,1.53,546.0,47,0.0
4,2.11,494.0,47,0.0
...,...,...,...,...
20042,358.49,0.0,0,171.0
20043,358.86,0.0,0,171.0
20044,359.21,0.0,0,171.0
20045,359.58,0.0,0,171.0


In [16]:
# Convert into cartesian coordinates and save to new CSV

# Convert angles from degrees to radians
theta = np.deg2rad(df["lidar_angle_deg"])   # vertical plane angle
phi = np.deg2rad(df["servo_angle_deg"])     # horizontal plane angle
r = df["lidar_dist_mm"]/1000                # radius in m

# Convert to cartesian coordinates
x = r * np.sin(theta) * np.cos(phi)  
y = r * np.sin(theta) * np.sin(phi)
z = -r * np.cos(theta)

# Store in new dataframe
cartesian_df = pd.DataFrame({
    "x": x,
    "y": y,
    "z": z 
})

# Save to CSV 
cartesian_df.to_csv("lidar_cartesian.csv", index=False)
print(f"Saved to CSV")

cartesian_df

Saved to CSV


,x,y,z
0,0.000000,0.0,-0.000000
1,0.000000,0.0,-0.000000
2,0.000000,0.0,-0.000000
3,0.014578,0.0,-0.545805
4,0.018188,0.0,-0.493665
...,...,...,...
20042,0.000000,-0.0,-0.000000
20043,0.000000,-0.0,-0.000000
20044,0.000000,-0.0,-0.000000
20045,0.000000,-0.0,-0.000000


In [12]:
# Optional: Make point cloud (tested with sample CSV)

# cartesian_df = pd.read_csv("/Users/anxiankhoo/Documents/lidar/spherical-data-livingRoom.csv")  # Replace with your filename if different
cartesian_df = pd.read_csv("/Users/anxiankhoo/Documents/lidar/lidar_cartesian.csv")

# Extract point coordinates as a NumPy array
points = cartesian_df[["x", "y", "z"]].to_numpy()

# Create a PyVista point cloud
point_cloud = pv.PolyData(points)
point_cloud["z"] = points[:, 2]

# Save as .glb for later
point_cloud.save("pointcloud.ply")

# Basic plot
pl = pv.Plotter()
pl.add_mesh(point_cloud, 
            render_points_as_spheres = True,
            scalars = points[:, 2],
            point_size = 2,
            show_scalar_bar = False,
            )
camera = pv.Camera()
pl.camera = camera
pl.camera_position = 'yz'
pl.window_size = [400, 400]
pl.camera.azimuth = 45
pl.add_axes()

pl.show()


Widget(value='<iframe src="http://localhost:57226/index.html?ui=P_0x1430df2f0_2&reconnect=auto" class="pyvista…

In [18]:
# Tracking change in floor points

# Imports
from datetime import datetime
import os 

# Definitions 
alert_threshold = -20 # Trigger alert if >20% decrease in floor space

# Detecting points on floor level (lowest 2 cm) 
min_z = cartesian_df["z"].min()
floor_band = 0.05  # 2 cm band
floor_points = df[cartesian_df["z"] <= (min_z + floor_band)]

# Count number of floor points 
floor_count = len(floor_points)

# Prepare entry with today's date and time 
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
entry = pd.DataFrame({"timestamp": [now], "floor_point_count": [floor_count]})

# Load into CSV 
summary_path = "floor_point_counts.csv"
if os.path.exists(summary_path):
    summary_df = pd.read_csv(summary_path)
    # Append new entry
    summary_df = pd.concat([summary_df, entry], ignore_index=True)
else:
    summary_df = entry

# Sort from oldest to newest
summary_df["timestamp"] = pd.to_datetime(summary_df["timestamp"]) 
summary_df = summary_df.sort_values("timestamp").reset_index(drop=True)

# Sort from oldest to newest
summary_df = summary_df.sort_values("timestamp").reset_index(drop=True)

# Compute percentage change and flag alert if needed 
summary_df["pct_change"] = summary_df["floor_point_count"].pct_change() * 100
summary_df["alert"] = summary_df["pct_change"] < alert_threshold

# Save updated summary
summary_df.to_csv(summary_path, index=False)
print(f"Saved {floor_count} floor points to {summary_path}")

Saved 94 floor points to floor_point_counts.csv


### Clutter detection: 

1. Reduction in floor space 
- track % decrease in number of floor points  
- floor point = point within 5cm of the min z value

2. Stack detection (voxel-based)
- track % change in number of tall tiles
- tall tiles = XY tiles with height higher than a defined threshold eg. 1.5m
- _voxelisation_ used to get XY tiles: divide the space into XY grid tiles (like 0.25m x 0.25m), and take max z for each tile as the height of the tile

In [20]:
from datetime import datetime
import os
import pandas as pd
import numpy as np

# === PARAMETERS ===
alert_floor_threshold = -20  # % decrease in floor points
alert_stack_threshold = 40   # % increase in tall stacks
stack_height_above_floor = 1.5  # meters
voxel_size = 0.25  # XY tile size in meters

# === LOAD LIDAR POINT CLOUD ===
cartesian_df = pd.read_csv("lidar_cartesian.csv")

# === FLOOR POINT DETECTION ===
min_z = cartesian_df["z"].min()
floor_band = 0.05  # 5cm band
floor_points = cartesian_df[cartesian_df["z"] <= (min_z + floor_band)]
floor_count = len(floor_points)

# === STACK DETECTION (VOXELIZED HEIGHT MAP) ===
# Bin by XY tiles
cartesian_df["x_bin"] = (cartesian_df["x"] / voxel_size).astype(int)
cartesian_df["y_bin"] = (cartesian_df["y"] / voxel_size).astype(int)

# Compute max height in each tile
tile_heights = cartesian_df.groupby(["x_bin", "y_bin"])["z"].max().reset_index()

# Count tiles that exceed stack threshold
tall_tiles = tile_heights[tile_heights["z"] >= (min_z + stack_height_above_floor)]
tall_tile_count = len(tall_tiles)

# === ENTRY FOR TODAY ===
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
entry = pd.DataFrame({
    "timestamp": [now],
    "floor_point_count": [floor_count],
    "tall_tile_count": [tall_tile_count]
})

# === LOAD INTO CSV & CALCULATE % CHANGES ===
summary_path = "summary.csv"
if os.path.exists(summary_path):
    summary_df = pd.read_csv(summary_path)
    summary_df = pd.concat([summary_df, entry], ignore_index=True)
else:
    summary_df = entry

# Sort by timestamp 
summary_df["timestamp"] = pd.to_datetime(summary_df["timestamp"])
summary_df = summary_df.sort_values("timestamp").reset_index(drop=True)

# Calculate % changes 
summary_df["floor_pct_change"] = summary_df["floor_point_count"].pct_change() * 100
summary_df["stack_pct_change"] = summary_df["tall_tile_count"].pct_change() * 100

# Set alert status (True/False)
summary_df["floor_alert"] = summary_df["floor_pct_change"] < alert_floor_threshold
summary_df["stack_alert"] = summary_df["stack_pct_change"] > alert_stack_threshold

# Save 
summary_df.to_csv(summary_path, index=False)

# === PRINT ===
print(f"✅ Saved {floor_count} floor points, {tall_tile_count} tall tiles to {summary_path}")
if summary_df.iloc[-1]["floor_alert"]:
    print("🚨 ALERT: Significant reduction in floor space detected!")
if summary_df.iloc[-1]["stack_alert"]:
    print("🚨 ALERT: Significant increase in tall clutter detected!")


✅ Saved 94 floor points, 32 tall tiles to summary.csv
